In [1]:
import xlrd
import xlwt
import pandas as pd
from pandas.core.frame import DataFrame
import numpy as np
from scipy.stats import zscore

In [2]:
# 打开文件
xlsx = xlrd.open_workbook('附件1 近5年402家供应商的相关数据.xlsx')
# 查看所有sheet列表
print('All sheets: %s' % xlsx.sheet_names())


All sheets: ['企业的订货量（m³）', '供应商的供货量（m³）']


In [3]:
sheet1 = xlsx.sheets()[0]    # 获得第1张sheet，索引从0开始
sheet1_name = sheet1.name    # 获得名称
sheet1_cols = sheet1.ncols   # 获得列数
sheet1_nrows = sheet1.nrows  # 获得行数
print('Sheet1 Name: %s\nSheet1 cols: %s\nSheet1 rows: %s' % (sheet1_name, sheet1_cols, sheet1_nrows))
sheet2 = xlsx.sheets()[1]    # 获得第1张sheet，索引从0开始
sheet2_name = sheet2.name    # 获得名称
sheet2_cols = sheet2.ncols   # 获得列数
sheet2_nrows = sheet2.nrows  # 获得行数
print('Sheet2 Name: %s\nSheet2 cols: %s\nSheet1 rows: %s' % (sheet2_name, sheet2_cols, sheet2_nrows))


Sheet1 Name: 企业的订货量（m³）
Sheet1 cols: 242
Sheet1 rows: 403
Sheet2 Name: 供应商的供货量（m³）
Sheet2 cols: 242
Sheet1 rows: 403


In [4]:
dic={}
sum_ding={}
average_ding={}
qiu_ding={}
max_ding={}

In [5]:
#转化订货量表
for i in range(sheet1.nrows-1):
            lis=[]
            sum_qiu=0
            for j in range(2,sheet1.ncols):
                lis.append(sheet1.cell(i+1,j).value)
                sum_qiu=sheet1.cell(i+1,j).value+sum_qiu
            dic[sheet1.cell(i+1,0).value]=lis
            #计算总的供应量
            sum_ding[sheet1.cell(i+1,0).value]=sum_qiu
            #计算供应量每周
            average=sum_qiu/240
            average_ding[sheet1.cell(i+1,0).value]=average
            #最大供应量
            max_ding[sheet1.cell(i+1,0).value]=max(lis)

In [6]:
ID=[]

In [7]:
#把商户信息提取出来
ID=list(dic.keys())

In [8]:
#把值提取出来转化成列表
sum_d=list(sum_ding.values())
ave_d=list(average_ding.values())
max_d=list(max_ding.values())

In [9]:
def write_excel_xls(path, sheet_name, value):
    index = len(value)  # 获取需要写入数据的行数
    workbook = xlwt.Workbook()  # 新建一个工作簿
    sheet = workbook.add_sheet(sheet_name)  # 在工作簿中新建一个表格
    for i in range(0, index):
        for j in range(0, len(value[i])):
            sheet.write(i, j, value[i][j])  # 像表格中写入数据（对应的行和列）
    workbook.save(path)  # 保存工作簿
    print("xls格式表格写入数据成功！")



In [10]:
dic_g={}
average_gong={}
qiu_gong={}
max_gong={}
sum_gong=[]

In [11]:
#转化sheet2
for i in range(sheet2.nrows-1):
            lis_1=[]
            sum_g=0
            qiu_ding=0
            for j in range(2,sheet2.ncols):
                lis_1.append(sheet2.cell(i+1,j).value)
                sum_g=sheet2.cell(i+1,j).value+sum_g
                qiu_ding=abs(sheet2.cell(i+1,j).value-sheet1.cell(i+1,j).value)+qiu_ding
            dic_g[sheet2.cell(i+1,0).value]=lis_1
            #计算总的供货量
            sum_gong.append(sum_g)
            #计算平均供货量
            average_g=sum_g/240
            average_gong[sheet2.cell(i+1,0).value]=average_g
            #计算订货量和供货量之间的差值的绝对值之和
            qiu_gong[sheet2.cell(i+1,0).value]=qiu_ding
            #最大供货量
            max_gong[sheet2.cell(i+1,0).value]=max(lis_1)

In [12]:
#把字典values提取出来
qiu_g=list(qiu_gong.values())
ave_g=list(average_gong.values())
max_g=list(max_gong.values())

In [13]:
#订平均减去供平均
ave=[ave_d[i]-ave_g[i]for i in range(0,len(ave_d))]

In [14]:
#订最大和供最大
max_dg=[max_d[i]-max_g[i]for i in range(0,len(max_d))]

In [15]:
#gong/ding
man_zu=[sum_gong[i]/sum_d[i]for i in range(0,len(sum_d))]


In [16]:
lis_key=['满足率','订-供','供货总量','订货总量','平均供货量','平均订货量','平均值相减','供货量最大值','订最大-供最大']

In [17]:
lis_value=[man_zu,qiu_g,sum_gong,sum_d,ave_g,ave_d,ave,max_g,max_dg]

In [18]:
data=DataFrame(lis_value)#这时候是以行为标准写入的
print(data)

          0           1             2           3            4           5    \
0    0.212121    0.883495      0.920092    0.089762     1.057204    0.064935   
1  202.000000  154.000000   2271.000000  669.000000   480.000000  448.000000   
2   49.000000  273.000000  13138.000000   64.000000  6912.000000   30.000000   
3  231.000000  309.000000  14279.000000  713.000000  6538.000000  462.000000   
4    0.204167    1.137500     54.741667    0.266667    28.800000    0.125000   
5    0.962500    1.287500     59.495833    2.970833    27.241667    1.925000   
6    0.758333    0.150000      4.754167    2.704167    -1.558333    1.800000   
7    6.000000   67.000000    387.000000    8.000000   128.000000    5.000000   
8   37.000000   -7.000000     53.000000   92.000000    12.000000   95.000000   

           6          7           8           9    ...        392         393  \
0     1.034544   0.436170    0.043357    0.295139  ...   0.458333    0.167832   
1  1736.000000  67.000000  688.000000

In [19]:
data=data.T#转置之后得到想要的结果


In [20]:
data.to_excel("1.xlsx", sheet_name="初步处理")


In [21]:
#主成分分析，对于9个指标进行降维到3个
r=np.corrcoef(data.T)#数据标准化并计算相关系数矩阵
print("相关系数阵为：\n", np.corrcoef(data.T))


相关系数阵为：
 [[ 1.         -0.05815765  0.20760418  0.14253788  0.20760418  0.14253788
  -0.06956186  0.05705161 -0.41341182]
 [-0.05815765  1.          0.41528647  0.68848067  0.41528647  0.68848067
   0.99954263  0.77836251  0.15868403]
 [ 0.20760418  0.41528647  1.          0.94563595  1.          0.94563595
   0.39625205  0.49598788 -0.09691195]
 [ 0.14253788  0.68848067  0.94563595  1.          0.94563595  1.
   0.67331476  0.67307314 -0.02025434]
 [ 0.20760418  0.41528647  1.          0.94563595  1.          0.94563595
   0.39625205  0.49598788 -0.09691195]
 [ 0.14253788  0.68848067  0.94563595  1.          0.94563595  1.
   0.67331476  0.67307314 -0.02025434]
 [-0.06956186  0.99954263  0.39625205  0.67331476  0.39625205  0.67331476
   1.          0.77258316  0.16313547]
 [ 0.05705161  0.77836251  0.49598788  0.67307314  0.49598788  0.67307314
   0.77258316  1.         -0.07181413]
 [-0.41341182  0.15868403 -0.09691195 -0.02025434 -0.09691195 -0.02025434
   0.16313547 -0.07181413  1.

In [22]:
d, e = np.linalg.eig(r) 

In [23]:
rate = d / d.sum()  # 计算各主成分的贡献率
print("特征值为：", d)
print("特征向量为：\n", e)
print("各主成分的贡献率为：", rate)

特征值为： [ 5.34239282e+00+0.00000000e+00j  1.74821117e+00+0.00000000e+00j
  1.06980295e+00+0.00000000e+00j  5.90459753e-01+0.00000000e+00j
  2.48902155e-01+0.00000000e+00j  2.31153688e-04+0.00000000e+00j
  6.16108128e-17+0.00000000e+00j -1.62431621e-16+4.36811227e-17j
 -1.62431621e-16-4.36811227e-17j]
特征向量为：
 [[ 5.90495738e-02+0.00000000e+00j -5.16453996e-01+0.00000000e+00j
   4.20070464e-01+0.00000000e+00j  7.43059021e-01+0.00000000e+00j
   3.41123935e-02+0.00000000e+00j  5.37205412e-03+0.00000000e+00j
   1.04276818e-16+0.00000000e+00j -9.23248365e-16+9.38947219e-16j
  -9.23248365e-16-9.38947219e-16j]
 [ 3.45100550e-01+0.00000000e+00j  3.69545741e-01+0.00000000e+00j
   2.97137965e-01+0.00000000e+00j  8.17458914e-02+0.00000000e+00j
  -3.26189273e-01+0.00000000e+00j -7.36864135e-01+0.00000000e+00j
  -5.07027919e-15+0.00000000e+00j  1.22520326e-13-1.32941773e-13j
   1.22520326e-13+1.32941773e-13j]
 [ 3.77785542e-01+0.00000000e+00j -2.74910961e-01+0.00000000e+00j
  -3.11750077e-01+0.00000000

In [29]:
d_1=[]#主成分特征值
rate_1=[]
e_1=[]
for i in range(len(d)):
    d_1.append(np.round(abs(d[i]),4))
    rate_1.append(np.round(abs(rate[i]),4))
    e_1.append(np.round(abs(e[i]),4))
print("特征值为：",d_1)
#print(rate_1)
print("各主成分的贡献率为：",rate_1)

特征值为： [5.3424, 1.7482, 1.0698, 0.5905, 0.2489, 0.0002, 0.0, 0.0, 0.0]
各主成分的贡献率为： [0.5936, 0.1942, 0.1189, 0.0656, 0.0277, 0.0, 0.0, 0.0, 0.0]


In [35]:
k = 5 # 提出主成分的个数
F = e_1[:, :k]
score_mat = data.dot(F)  # 计算主成分得分矩阵
score1 = score_mat.dot(rate_1[0:k])  # 计算各评价对象的得分

In [36]:
score1

0       158.711580
1       246.906463
2      9221.656991
3       468.792828
4      4307.363936
          ...     
397     142.052961
398     228.606249
399     536.621153
400     153.764188
401     616.368336
Length: 402, dtype: float64

In [37]:
index = score1.argsort()  # 排序后的每个元素在原数组中的位置
print("从各个供应商的编号排序为：", index)

从各个供应商的编号排序为： 0       25
1      205
2      304
3       32
4       43
      ... 
397    150
398    360
399    228
400    200
401    139
Length: 402, dtype: int64


In [214]:
score1.to_excel("2.xlsx", sheet_name="供应商得分")